### Библиотеки / данные 

импортируем numpy и pandas

In [1]:
import numpy as np
import pandas as pd

настройки pandas

In [2]:
pd.options.display.max_rows = 10

- считываем данные
- используем столбец Symbol в качестве индекса 
- считываем только столбцы ['Symbol', 'Sector', 'Price', 'Book Value']

| Column Name        | Description
| ------------- |:-------------:|
|Symbol|Сокращенное название организации|
|Name|Полное название организации|
|Sector|Сектор экономики|
|Price|Стоимость акции|
|Dividend Yield|Дивидендная доходность|
|Price/Earnings|Цена / прибыль|
|Earnings/Share|Прибыль на акцию|
|Book Value|Балансовая стоимость компании|
|52 week low|52-недельный минимум|
|52 week high|52-недельный максимум|
|Market Cap|Рыночная капитализация|
|EBITDA|**E**arnings **b**efore **i**nterest, **t**axes, **d**epreciation and **a**mortization|
|Price/Sales|Цена / объём продаж|
|Price/Book|Цена / балансовая стоимость|
|SEC Filings|Ссылка *sec.gov*|

In [3]:
sp500 = pd.read_csv("../data/sp500.csv",
                    index_col='Symbol', 
                    usecols=['Symbol', 'Sector', 'Price', 'Book Value'])

### Мотивация

создаем DataFame для примера

In [4]:
np.random.seed(123)
df = pd.DataFrame({'value':np.random.random(10000), 'key':range(100, 10100)})
df.head()

,value,key
0,0.696469,100
1,0.286139,101
2,0.226851,102
3,0.551315,103
4,0.719469,104


отбираем строку, в котором значение столбца key равно 10099

In [5]:
df[df.key==10099]

,value,key
9999,0.613297,10099


измеряем время выполнения операции отбора

In [6]:
%timeit df[df.key==10099]

390 µs ± 13.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


превращаем столбец key в index

In [7]:
df_with_index = df.set_index(['key'])
df_with_index.head()

,value
key,
100,0.696469
101,0.286139
102,0.226851
103,0.551315
104,0.719469


теперь можно найти это значение с помощью индекса

In [8]:
df_with_index.loc[10099]

value    0.613297
Name: 10099, dtype: float64

и теперь операция выполняется намного быстрее

In [9]:
%timeit df_with_index.loc[10099]

79.9 µs ± 2.13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


<center><font color = 'green'><b>Вывод: использование индекса повышает скорость доступа к данным более чем в 5.9 раз!</b></font>

### Операции

#### сброс

исследуем несколько строк датафрейма data

In [ ]:
sp500.head(3)

сбрасываем индекс, помещая значения индекса в столбец

In [ ]:
index_moved_to_col = sp500.reset_index()
index_moved_to_col.head()

#### установка

а теперь делаем столбец Sector индексом

In [ ]:
index_moved_to_col.set_index('Sector').head()

#### над множествами

Датафреймы для примера:

In [ ]:
data_rnd_part_1 = sp500.sample(100, random_state=333)
data_rnd_part_2 = sp500.sample(100, random_state=777)

In [ ]:
data_rnd_part_1.head()

In [ ]:
data_rnd_part_2.head()

объединение:

In [ ]:
ind_union = data_rnd_part_1.index | data_rnd_part_2.index
ind_union

пересечение

In [ ]:
ind_intersection = data_rnd_part_1.index & data_rnd_part_2.index
len(ind_intersection)

разность

In [ ]:
ind_diff = data_rnd_part_1.index.difference(data_rnd_part_2.index)
len(ind_diff)

### Иерархическая индексация

сначала помещаем символы в столбец

In [ ]:
reindexed = sp500.reset_index()

In [ ]:
reindexed.head()

а теперь индексируем датафрейм data по столбцам Sector и Symbol

In [ ]:
multi_fi = reindexed.set_index(['Sector', 'Symbol'])
multi_fi.head()

наш индекс - это MultiIndex

In [ ]:
type(multi_fi.index)

он имеет два уровня

In [ ]:
len(multi_fi.index.levels)

каждый уровень индекса - это индекс

In [ ]:
multi_fi.index.levels[1]

изменение порядка уровней индекса:

In [ ]:
multi_fi.reorder_levels([1, 0], axis=0).head()

получаем все акции, которые имеют значение Industrials <br> обратите внимание, что в результатах индекс уровня 0 не выводится 

In [ ]:
multi_fi.xs('Industrials').head()

отбираем строки, в которых индекс уровня 1 (Symbol) имеет значение ALLE

In [ ]:
multi_fi.xs('ALLE', level=1)

скомбинируем уровни индексов

In [ ]:
multi_fi.xs('Industrials').xs('UPS')

комбинируем уровни индексов, используя кортеж

In [ ]:
multi_fi.xs(('Industrials', 'UPS'))

### [Типы](http://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#index-types)

#### Основной тип Index

покажем, что столбцы фактически являются индексом

In [ ]:
sp500.columns

In [ ]:
sp500.index

#### Int64Index и RangeIndex

Явно создаем Int64Index

In [ ]:
df_i64 = pd.DataFrame(np.arange(10, 20), index=np.arange(0, 10))
df_i64.head()

смотрим индекс

In [ ]:
df_i64.index

по умолчанию мы получаем RangeIndex

In [ ]:
df_range = pd.DataFrame(np.arange(10, 15))
df_range.head()

In [ ]:
df_range.index

#### Float64Index

индексы, использующие Float64Index

In [ ]:
df_f64 = pd.DataFrame(np.arange(0, 1000, 5), 
                      np.arange(0.0, 100.0, 0.5))
df_f64.head()

In [ ]:
df_f64.index